# Phishing Website Detection 

Step -1 : Data preprocessing 

This dataset contains few website links (Some of them are legitimate websites and a few are fake websites)

Pre-Processing the data before building a model and also Extracting the features from the data based on certain conditions

In [1]:
#importing numpy and pandas which are required for data pre-processing
import numpy as np
import pandas as pd

In [2]:
#Loading the data
raw_data = pd.read_csv("dataset2.csv") 

In [3]:
raw_data.head()

,URL,Target
0,https://locking-app-adverds.000webhostapp.com/...,yes
1,http://www.myhealthcarepharmacy.ca/wp-includes...,yes
2,http://code.google.com/p/pylevenshtein/,no
3,http://linkedin.com/,no
4,http://imageshack.com/f/219/cadir2yr3.jpg,no


We need to split the data according to parts of the URL

A typical URL could have the form http://www.example.com/index.html, which indicates a protocol (http), a hostname (www.example.com), and a file name (index.html).

In [4]:
raw_data['URL'].str.split("://").head() #Here we divided the protocol from the entire URL. but need it to be divided it 
                                                 #seperate column

0    [https, locking-app-adverds.000webhostapp.com/...
1    [http, www.myhealthcarepharmacy.ca/wp-includes...
2             [http, code.google.com/p/pylevenshtein/]
3                                [http, linkedin.com/]
4           [http, imageshack.com/f/219/cadir2yr3.jpg]
Name: URL, dtype: object

In [5]:
seperation_of_protocol = raw_data['URL'].str.split("://",expand = True) #expand argument in the split method will give you a new column

In [56]:
seperation_of_protocol.head()

,0,1,2,3,4,5,6
0,https,locking-app-adverds.000webhostapp.com/payment-...,None,None,None,None,None
1,http,www.myhealthcarepharmacy.ca/wp-includes/js/jqu...,None,None,None,None,None
2,http,code.google.com/p/pylevenshtein/,None,None,None,None,None
3,http,linkedin.com/,None,None,None,None,None
4,http,imageshack.com/f/219/cadir2yr3.jpg,None,None,None,None,None


In [7]:
type(seperation_of_protocol)

pandas.core.frame.DataFrame

In [8]:
seperation_domain_name = seperation_of_protocol[1].str.split("/",1,expand = True) #split(seperator,no of splits according to seperator(delimiter),expand)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_7936\3656433041.py:1: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  seperation_domain_name = seperation_of_protocol[1].str.split("/",1,expand = True) #split(seperator,no of splits according to seperator(delimiter),expand)


In [9]:
type(seperation_domain_name)

pandas.core.frame.DataFrame

In [57]:
seperation_domain_name.columns=["domain_name","address"] #renaming columns of data frame

In [12]:
#Concatenation of data frames
splitted_data = pd.concat([seperation_of_protocol[0],seperation_domain_name],axis=1)


In [13]:
splitted_data.columns = ['protocol','domain_name','address']

In [14]:
splitted_data.head()

,protocol,domain_name,address
0,https,locking-app-adverds.000webhostapp.com,payment-update-0.html?fb_source=bookmark_apps&...
1,http,www.myhealthcarepharmacy.ca,wp-includes/js/jquery/ini.php
2,http,code.google.com,p/pylevenshtein/
3,http,linkedin.com,
4,http,imageshack.com,f/219/cadir2yr3.jpg


In [15]:
splitted_data['is_phished'] = pd.Series(raw_data['Target'], index=splitted_data.index)

In [16]:
splitted_data

,protocol,domain_name,address,is_phished
0,https,locking-app-adverds.000webhostapp.com,payment-update-0.html?fb_source=bookmark_apps&...,yes
1,http,www.myhealthcarepharmacy.ca,wp-includes/js/jquery/ini.php,yes
2,http,code.google.com,p/pylevenshtein/,no
3,http,linkedin.com,,no
4,http,imageshack.com,f/219/cadir2yr3.jpg,no
...,...,...,...,...
1776,https,docs.google.com,document/u/1/,no
1777,http,www.charlestodd.com,wp-includes/pomo/adobe2.html,yes
1778,http,tslimpact.com,medsynaptic/wp-content/themes/twentyseventeen/...,yes
1779,http,daoudilorin11.mystagingwebsite.com,wp-content/plugins/ubh/acc/dir/68e1c/dir/car.php,yes


Domain name column can be further sub divided into domain_names as well as sub_domain_names 

Similarly, address column can also be further sub divided into path,query_string,file..................

In [17]:
type(splitted_data)

pandas.core.frame.DataFrame

### Features Extraction


Feature-1

1.Long URL to Hide the Suspicious Part

If the length of the URL is greater than or equal 54 characters then the URL classified as phishing


0 --- indicates legitimate

1 --- indicates Phishing

2 --- indicates Suspicious

In [18]:
def long_url(l):
    l= str(l)
    """This function is defined in order to differntiate website based on the length of the URL"""
    if len(l) < 54:
        return 0
    elif len(l) >= 54 and len(l) <= 75:
        return 2
    return 1

In [19]:
#Applying the above defined function in order to divide the websites into 3 categories
splitted_data['long_url'] = raw_data['URL'].apply(long_url) 


In [20]:
#Will show the results only the websites which are legitimate according to above condition as 0 is legitimate website
splitted_data[splitted_data.long_url == 0] 

,protocol,domain_name,address,is_phished,long_url
2,http,code.google.com,p/pylevenshtein/,no,0
3,http,linkedin.com,,no,0
4,http,imageshack.com,f/219/cadir2yr3.jpg,no,0
6,http,www.7-zip.org,download.html,no,0
7,http,ebay.com,,no,0
...,...,...,...,...,...
1766,http,malomolk.com,nab/cardinfo.html,yes,0
1774,http,www.dpincsupport.com,,no,0
1775,https,bitcoin.org,en/,no,0
1776,https,docs.google.com,document/u/1/,no,0


Feature-2

2.URL’s having “@” Symbol

Using “@” symbol in the URL leads the browser to ignore everything preceding the “@” symbol and the real address often follows the “@” symbol.

IF {Url Having @ Symbol→ Phishing
    Otherwise→ Legitimate }


0 --- indicates legitimate

1 --- indicates Phishing


In [21]:
def have_at_symbol(l):
    """This function is used to check whether the URL contains @ symbol or not"""
    if "@" in str(l):
        return 1
    return 0
    

In [22]:
splitted_data['having_@_symbol'] = raw_data['URL'].apply(have_at_symbol)

In [23]:
splitted_data

,protocol,domain_name,address,is_phished,long_url,having_@_symbol
0,https,locking-app-adverds.000webhostapp.com,payment-update-0.html?fb_source=bookmark_apps&...,yes,1,0
1,http,www.myhealthcarepharmacy.ca,wp-includes/js/jquery/ini.php,yes,2,0
2,http,code.google.com,p/pylevenshtein/,no,0,0
3,http,linkedin.com,,no,0,0
4,http,imageshack.com,f/219/cadir2yr3.jpg,no,0,0
...,...,...,...,...,...,...
1776,https,docs.google.com,document/u/1/,no,0,0
1777,http,www.charlestodd.com,wp-includes/pomo/adobe2.html,yes,2,0
1778,http,tslimpact.com,medsynaptic/wp-content/themes/twentyseventeen/...,yes,1,0
1779,http,daoudilorin11.mystagingwebsite.com,wp-content/plugins/ubh/acc/dir/68e1c/dir/car.php,yes,1,0


Feature-3

3.Redirecting using “//”

The existence of “//” within the URL path means that the user will be redirected to another website.
An example of such URL’s is: “http://www.legitimate.com//http://www.phishing.com”. 
We examine the location where the “//” appears. 
We find that if the URL starts with “HTTP”, that means the “//” should appear in the sixth position. 
However, if the URL employs “HTTPS” then the “//” should appear in seventh position.

IF {ThePosition of the Last Occurrence of "//" in the URL > 7→ Phishing
    
    Otherwise→ Legitimate

0 --- indicates legitimate

1 --- indicates Phishing


In [24]:
def redirection(l):
    """If the url has symbol(//) after protocol then such URL is to be classified as phishing """
    if "//" in str(l):
        return 1
    return 0

In [25]:
splitted_data['redirection_//_symbol'] = seperation_of_protocol[1].apply(redirection)

In [26]:
splitted_data.head()

,protocol,domain_name,address,is_phished,long_url,having_@_symbol,redirection_//_symbol
0,https,locking-app-adverds.000webhostapp.com,payment-update-0.html?fb_source=bookmark_apps&...,yes,1,0,0
1,http,www.myhealthcarepharmacy.ca,wp-includes/js/jquery/ini.php,yes,2,0,0
2,http,code.google.com,p/pylevenshtein/,no,0,0,0
3,http,linkedin.com,,no,0,0,0
4,http,imageshack.com,f/219/cadir2yr3.jpg,no,0,0,0


Feature-4

4.Adding Prefix or Suffix Separated by (-) to the Domain

The dash symbol is rarely used in legitimate URLs. Phishers tend to add prefixes or suffixes separated by (-) to the domain name
so that users feel that they are dealing with a legitimate webpage. 

For example http://www.Confirme-paypal.com/.
    
IF {Domain Name Part Includes (−) Symbol → Phishing
    
    Otherwise → Legitimate
    
1 --> indicates phishing

0 --> indicates legitimate
    

In [27]:
def prefix_suffix_seperation(l):
    if '-' in str(l):
        return 1
    return 0

In [28]:
splitted_data['prefix_suffix_seperation'] = seperation_domain_name['domain_name'].apply(prefix_suffix_seperation)

In [29]:
splitted_data.head()

,protocol,domain_name,address,is_phished,long_url,having_@_symbol,redirection_//_symbol,prefix_suffix_seperation
0,https,locking-app-adverds.000webhostapp.com,payment-update-0.html?fb_source=bookmark_apps&...,yes,1,0,0,1
1,http,www.myhealthcarepharmacy.ca,wp-includes/js/jquery/ini.php,yes,2,0,0,0
2,http,code.google.com,p/pylevenshtein/,no,0,0,0,0
3,http,linkedin.com,,no,0,0,0,0
4,http,imageshack.com,f/219/cadir2yr3.jpg,no,0,0,0,0


Feature - 5

5. Sub-Domain and Multi Sub-Domains

The legitimate URL link has two dots in the URL since we can ignore typing “www.”. 
If the number of dots is equal to three then the URL is classified as “Suspicious” since it has one sub-domain.
However, if the dots are greater than three it is classified as “Phishy” since it will have multiple sub-domains

0 --- indicates legitimate

1 --- indicates Phishing

2 --- indicates Suspicious


In [30]:
def sub_domains(l):
    l= str(l)
    if l.count('.') < 3:
        return 0
    elif l.count('.') == 3:
        return 2
    return 1

In [31]:
splitted_data['sub_domains'] = splitted_data['domain_name'].apply(sub_domains)

In [32]:
splitted_data

,protocol,domain_name,address,is_phished,long_url,having_@_symbol,redirection_//_symbol,prefix_suffix_seperation,sub_domains
0,https,locking-app-adverds.000webhostapp.com,payment-update-0.html?fb_source=bookmark_apps&...,yes,1,0,0,1,0
1,http,www.myhealthcarepharmacy.ca,wp-includes/js/jquery/ini.php,yes,2,0,0,0,0
2,http,code.google.com,p/pylevenshtein/,no,0,0,0,0,0
3,http,linkedin.com,,no,0,0,0,0,0
4,http,imageshack.com,f/219/cadir2yr3.jpg,no,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
1776,https,docs.google.com,document/u/1/,no,0,0,0,0,0
1777,http,www.charlestodd.com,wp-includes/pomo/adobe2.html,yes,2,0,0,0,0
1778,http,tslimpact.com,medsynaptic/wp-content/themes/twentyseventeen/...,yes,1,0,0,0,0
1779,http,daoudilorin11.mystagingwebsite.com,wp-content/plugins/ubh/acc/dir/68e1c/dir/car.php,yes,1,0,0,0,0


### Classification of URLs using Random forest 

In [48]:
from sklearn.ensemble._forest import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score

In [34]:
#Features
x = splitted_data.columns[4:9]
x   

Index(['long_url', 'having_@_symbol', 'redirection_//_symbol',
       'prefix_suffix_seperation', 'sub_domains'],
      dtype='object')

In [35]:
#variable to be predicted; yes = 0 and no = 1
y = pd.factorize(splitted_data['is_phished'])[0]
y 

array([0, 0, 1, ..., 0, 0, 0], dtype=int64)

In [36]:
# Create a random forest Classifier. By convention, clf means 'Classifier'
clf = RandomForestClassifier(n_estimators=100,n_jobs=2,random_state=0)

# Train the Classifier to take the training features and learn how they relate
# to the training y (the species)
clf.fit(splitted_data[x], y)

RandomForestClassifier(n_jobs=2, random_state=0)

In [37]:
test_data = pd.read_csv("dataset3.csv") 

In [51]:
import pickle
pickle.dump(clf, open('finalized_model.sav', 'wb'))

In [38]:
clf.predict(test_data[x]) #testing the classifier on test data.

array([0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1,
       0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1,

In [39]:
clf.predict_proba(test_data[x])[0:10] #predicted probability for each class.

array([[0.88800564, 0.11199436],
       [0.50510239, 0.49489761],
       [0.32367919, 0.67632081],
       [0.50510239, 0.49489761],
       [0.32367919, 0.67632081],
       [0.32367919, 0.67632081],
       [0.53582536, 0.46417464],
       [0.32367919, 0.67632081],
       [0.32367919, 0.67632081],
       [0.32367919, 0.67632081]])

### Evaluating classifier

In [60]:
preds = test_data.is_phished[clf.predict(test_data[x])] #predicted values

In [41]:
preds.head(10)

0    yes
0    yes
1     no
0    yes
1     no
1     no
0    yes
1     no
1     no
1     no
Name: is_phished, dtype: object

In [42]:
actual = pd.Series(test_data['is_phished']) #actual values

In [43]:
confusion_matrix(actual,preds) 

array([[ 94,  80],
       [ 64, 152]], dtype=int64)

In [44]:
accuracy_score(actual,preds) #accuracy of classifier

0.6307692307692307

In [45]:
#importance of features
list(zip(splitted_data[x], clf.feature_importances_))

[('long_url', 0.3567302891440939),
 ('having_@_symbol', 0.012172952111729666),
 ('redirection_//_symbol', 0.005264801542431981),
 ('prefix_suffix_seperation', 0.5692813985568205),
 ('sub_domains', 0.056550558644924025)]